In [0]:
# Importe MLflow, ketas y tensorflow
import mlflow 
import mlflow.keras
import keras
import tensorflow as tf
import tensorflow.keras as tk
from keras import models
from keras import layers
from tensorflow.keras.layers import Dense

In [0]:
# Usaremos argparse para pasarle argumentos a las funciones de entrenamiento
import argparse

parser = argparse.ArgumentParser(description='Entrenamiento de una red feed-forward para el problema de clasificación con datos icfes en TensorFlow/Keras')
parser.add_argument('--batch_size', '-b', type=int, default=32)
parser.add_argument('--epochs', '-e', type=int, default=5)
parser.add_argument('--learning_rate', '-l', type=float, default=0.05)
parser.add_argument('--num_hidden_units', '-n', type=int, default=8)
parser.add_argument('--num_hidden_layers', '-N', type=int, default=1)
parser.add_argument('--dropout', '-d', type=float, default=0.25)
parser.add_argument('--momentum', '-m', type=float, default=0.85)

_StoreAction(option_strings=['--momentum', '-m'], dest='momentum', nargs=None, const=None, default=0.85, type=<class 'float'>, choices=None, required=False, help=None, metavar=None)

In [0]:
args = parser.parse_args([])

In [0]:
def get_optimizer():
    """
    :return: Keras optimizer
    """
    optimizer = keras.optimizers.SGD(learning_rate=args.learning_rate,momentum=args.momentum, nesterov=True)
    return optimizer

In [0]:
# Obtenemos el dataset icfes

import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar archivo CSV
data_spark = spark.read.option("header","true").csv("/FileStore/tables/Icfes_limpio3-2.csv")
data = data_spark.toPandas()
# Codificación de variables categóricas
data_encoded = pd.get_dummies(data, columns=['cole_depto_ubicacion', 'cole_area_ubicacion','cole_naturaleza','cole_bilingue','fami_estratovivienda','presento_todas_las_areas'])
# Separar variables independientes (X) y dependientes (y)
X = data_encoded.drop('NIVEL',axis=1)
Y = data_encoded['NIVEL']
# Escalado de variables numéricas
scaler = StandardScaler()
X = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split

# Dividir en conjunto de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [0]:
# Esta función define una corrida del modelo, con entrenamiento y 
# registro en MLflow
def run_mlflow(run_name="MLflow icfes"):
    # Iniciamos una corrida de MLflow
    mlflow.start_run(run_name=run_name)
    run = mlflow.active_run()
    # MLflow asigna un ID al experimento y a la corrida
    experimentID = run.info.experiment_id
    runID = run.info.run_uuid
    # reistro automáticos de las métricas de keras
    mlflow.keras.autolog()
    model = models.Sequential()  
    # La primera capa de la red
    #model.add(Dense(53, input_dim=X.shape[1], activation='relu'))   
    model.add(layers.Flatten(input_shape=x_train[0].shape))
    # Agregamos capas ocultas a la red
    # en los argumentos: --num_hidden_layers o -N 
    for n in range(0, args.num_hidden_layers):
        # agregamos una capa densa (completamente conectada) con función de activación relu
        model.add(layers.Dense(args.num_hidden_units, activation=tf.nn.relu))
        # agregamos dropout como método de regularización para aleatoriamente descartar una capa
        # si los gradientes son muy pequeños
        model.add(layers.Dropout(args.dropout))
        # capa final con 1 nodos de salida y activación softmax 
        model.add(layers.Dense(1, activation=tf.nn.softmax))
        # Use 
        # https://keras.io/optimizers/
        optimizer = get_optimizer()

    # compilamos el modelo y definimos la función de pérdida  
    # otras funciones de pérdida comunes para problemas de clasificación
    # 1. sparse_categorical_crossentropy
    # 2. binary_crossentropy
    model.compile(optimizer=optimizer,
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    # entrenamos el modelo
    print("-" * 100)
    model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.batch_size)
    # evaluamos el modelo
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    mlflow.end_run(status='FINISHED')
    return (experimentID, runID)

In [0]:
x_train[0].shape

(53,)

In [0]:
# corrida con parámetros diferentes a los por defecto
args = parser.parse_args(["--batch_size", '256', '--epochs', '8'])
(experimentID, runID) = run_mlflow()
print("MLflow Run completed with run_id {} and experiment_id {}".format(runID, experimentID))
print(tf.__version__)
print("-" * 100)

----------------------------------------------------------------------------------------------------


2025/05/25 20:09:25 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/8


---------------------------------------------------------------------------
UnimplementedError                        Traceback (most recent call last)
File <command-2246144502270323>, line 3
      1 # corrida con parámetros diferentes a los por defecto
      2 args = parser.parse_args(["--batch_size", '256', '--epochs', '8'])
----> 3 (experimentID, runID) = run_mlflow()
      4 print("MLflow Run completed with run_id {} and experiment_id {}".format(runID, experimentID))
      5 print(tf.__version__)

File <command-2246144502270322>, line 40, in run_mlflow(run_name)
     38 # entrenamos el modelo
     39 print("-" * 100)
---> 40 model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.batch_size)
     41 # evaluamos el modelo
     42 test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

File /databricks/python/lib/python3.10/site-packages/mlflow/utils/autologging_utils/safety.py:550, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    540 try_log_autologgi

In [0]:
mlflow.end_run()